# Instructions
You will be peforming an ETL process:
* Extract the data from S3
* Perform transformations to clean up the data 
* Perform analysis to create 4 datasets required by the business 
* Load the results into Snowflake

### Extract the data from source 
* Read the `accidents_2017_to_2023_english_csv` from the S3 bucket in `techcatalyst-raw/accidents`

### Renaming
__Rename Columns__: For better readability and usability, rename the columns to be more descriptive and follow a consistent naming convention. 
* `inverse_data` --> `accident_date`
* `week_day` --> `day_of_week` 
* `hour` --> `accident_time` 
* `state` --> `state_code` 
* `road_id` --> `highway_number` 
* `city` --> `city_name` 
* `cause_of_accident` --> `primary_cause` 
* `type_of_accident` --> `accident_type` 
* `victims_condition` --> `casualty_status` 
* `weather_timestamp` --> `daylight_condition` 
* `road_direction` --> `traffic_direction` 
* `wheather_condition` --> `weather_condition`   
* `road_type` --> `highway_type` 
* `road_delineation` --> `road_geometry` 
* `people` --> `total_people_involved` 
* `deaths` --> `fatalities` 
* `slightly_injured` --> `minor_injuries` 
* `severely_injured` --> `serious_injuries` 
* `uninjured` --> `uninjured_count` 
* `ignored` --> `unknown_status` 
* `total_injured` --> `total_casualties` 
* `vehicles_involved` --> `vehicle_count` 
* `latitude` --> `accident_latitude` 
* `longitude` --> `accident_longitude` 
* `regional` --> `regional_office` 
* `police_station` --> `reporting_station`

### Fix data types and inconsistencies
* __Date/Time__: Convert `accident_date` to a proper datetime format. Standardize `hour` to datetime or extract hour as an integer for easier analysis.
* __Numeric Conversion__: Convert the km column to a numeric float type. This requires handling the comma decimal separator by replacing it with a period (.) before conversion.
    * Convert `km` to numeric (handle mixed formats like "43,4" vs 320)
* __Text Fields__: Standardize all object/string columns (e.g., `accident_cause`, `city`, `road_direction`) by converting them to lowercase and stripping leading/trailing whitespace. This prevents issues with duplicate categories (e.g., "sunny" vs. "Sunny").

### Data Quaulity (DQ) Checks 
Before analysis, it's vital to assess the data's integrity.

### Handle missing values
* Document which columns contain missing data. Count missing values in each column and document which ones have gaps 
* For simplicity in this project, remove rows where these key columns have missing data using
### Outliers
*  Examine numerical fields and look for values that seem physically impossible or are extreme statistical outliers (e.g., an accident involving 100 vehicles). `Boxplots` are a great way to visualize these. Document interesting findings.
* Document these columns 
### Data Validation (DQ)
* Create a new boolean column `total_injured_match_ind` to verify if the sum of `slightly_injured` and `severely_injured` equals the `total_injured` column

### Feature Engieering
* __Date Components__: Extract year, month, day, and quarter from the `accident_date`.
* __Weekend vs. Weekday__: Create a `weekend_flag` (True/False) based on the day_of_week.
* Time of Day Category: Create a time_of_day_category column based on the hour:
    * Morning (5am-12pm)
    * Afternoon (12pm-5pm)
    * Evening (5pm-9pm)
    * Night (9pm-5am)
* __Fatality & Vehicle Flags__:
    * `fatal_accident_flag`: True if deaths > 0
    * `multi_vehicle_flag`: True if vehicles_involved > 1
* __Accident Severity Score__: Create the accident_severity_score using the specified formula to quantify the overall severity of an accident:
    * `accident_severity_score` = (deaths * 10) + (severely_injured * 3) + slightly_injured

In [38]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from datetime import datetime
import numpy as np
from dotenv import load_dotenv

load_dotenv()



True

In [39]:
def load_accident_data():
    # YOUR CODE HERE: Load the CSV file into a DataFrame
    df = pd.read_csv('s3://techcatalyst-raw/accidents/accidents_2017_to_2023_english.csv')
    print(f"Loaded {len(df)} records with {len(df.columns)} columns")
    
    return df

df = load_accident_data()

Loaded 463152 records with 27 columns


In [40]:
df.head(5)

,inverse_data,week_day,hour,state,road_id,km,city,cause_of_accident,type_of_accident,victims_condition,...,slightly_injured,severely_injured,uninjured,ignored,total_injured,vehicles_involved,latitude,longitude,regional,police_station
0,2017-01-01,sunday,01:45:00,RS,116.0,"34,9",VACARIA,Mechanical loss/defect of vehicle,Rear-end collision,With injured victims,...,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS
1,2017-01-01,sunday,01:00:00,PR,376.0,636,TIJUCAS DO SUL,Incompatible velocity,Run-off-road,With dead victims,...,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR
2,2017-01-01,sunday,04:40:00,BA,101.0,65,ENTRE RIOS,Driver was sleeping,Head-on collision,With dead victims,...,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA
3,2017-01-01,sunday,06:30:00,PA,316.0,"72,5",CASTANHAL,Driver's lack of attention to conveyance,Side impact collision,With dead victims,...,0,0,3,0,0,3,-1.28998,-47.83483,SPRF-PA,DEL01-PA
4,2017-01-01,sunday,09:00:00,GO,20.0,"220,5",POSSE,Road's defect,Collision with fixed object,With injured victims,...,2,1,0,0,3,1,-14.14221,-46.32259,SPRF-DF,DEL02-DF


In [41]:
df.columns

Index(['inverse_data', 'week_day', 'hour', 'state', 'road_id', 'km', 'city',
       'cause_of_accident', 'type_of_accident', 'victims_condition',
       'weather_timestamp', 'road_direction', 'wheather_condition',
       'road_type', 'road_delineation', 'people', 'deaths', 'slightly_injured',
       'severely_injured', 'uninjured', 'ignored', 'total_injured',
       'vehicles_involved', 'latitude', 'longitude', 'regional',
       'police_station'],
      dtype='object')

In [42]:
def rename_columns(df):
    """Rename columns for better readability and consistency"""
    column_mapping = {
        # YOUR CODE HERE: Define the mapping of old column names to new column names'
        'inverse_data' : 'accident_date',
        'week_day' : 'day_of_week',
        'hour' : 'accident_time' ,
        'state' : 'state_code' ,
        'road_id' : 'highway_number' ,
        'city' : 'city_name' ,
        'cause_of_accident' : 'primary_cause' ,
        'type_of_accident' : 'accident_type' ,
        'victims_condition' : 'casualty_status' ,
        'weather_timestamp' : 'daylight_condition' ,
        'road_direction' : 'traffic_direction' ,
        'wheather_condition' : 'weather_condition'   ,
        'road_type' : 'highway_type' ,
        'road_delineation' : 'road_geometry' ,
        'people' : 'total_people_involved' ,
        'deaths' : 'fatalities' ,
        'slightly_injured' : 'minor_injuries' ,
        'severely_injured' : 'serious_injuries' ,
        'uninjured' : 'uninjured_count' ,
        'ignored' : 'unknown_status' ,
        'total_injured' : 'total_casualties' ,
        'vehicles_involved' : 'vehicle_count' ,
        'latitude' : 'accident_latitude' ,
        'longitude' : 'accident_longitude' ,
        'regional' : 'regional_office' ,
        'police_station' : 'reporting_station',

    }
    
    df_renamed = df.rename(columns=column_mapping)
    print("✓ Columns renamed successfully")
    return df_renamed

In [43]:
df_renamed = rename_columns(df)
df_renamed.head(5)

✓ Columns renamed successfully


,accident_date,day_of_week,accident_time,state_code,highway_number,km,city_name,primary_cause,accident_type,casualty_status,...,minor_injuries,serious_injuries,uninjured_count,unknown_status,total_casualties,vehicle_count,accident_latitude,accident_longitude,regional_office,reporting_station
0,2017-01-01,sunday,01:45:00,RS,116.0,"34,9",VACARIA,Mechanical loss/defect of vehicle,Rear-end collision,With injured victims,...,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS
1,2017-01-01,sunday,01:00:00,PR,376.0,636,TIJUCAS DO SUL,Incompatible velocity,Run-off-road,With dead victims,...,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR
2,2017-01-01,sunday,04:40:00,BA,101.0,65,ENTRE RIOS,Driver was sleeping,Head-on collision,With dead victims,...,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA
3,2017-01-01,sunday,06:30:00,PA,316.0,"72,5",CASTANHAL,Driver's lack of attention to conveyance,Side impact collision,With dead victims,...,0,0,3,0,0,3,-1.28998,-47.83483,SPRF-PA,DEL01-PA
4,2017-01-01,sunday,09:00:00,GO,20.0,"220,5",POSSE,Road's defect,Collision with fixed object,With injured victims,...,2,1,0,0,3,1,-14.14221,-46.32259,SPRF-DF,DEL02-DF


In [44]:
df_renamed.columns

Index(['accident_date', 'day_of_week', 'accident_time', 'state_code',
       'highway_number', 'km', 'city_name', 'primary_cause', 'accident_type',
       'casualty_status', 'daylight_condition', 'traffic_direction',
       'weather_condition', 'highway_type', 'road_geometry',
       'total_people_involved', 'fatalities', 'minor_injuries',
       'serious_injuries', 'uninjured_count', 'unknown_status',
       'total_casualties', 'vehicle_count', 'accident_latitude',
       'accident_longitude', 'regional_office', 'reporting_station'],
      dtype='object')

In [45]:
df_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463152 entries, 0 to 463151
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   accident_date          463152 non-null  object 
 1   day_of_week            463152 non-null  object 
 2   accident_time          463152 non-null  object 
 3   state_code             463152 non-null  object 
 4   highway_number         462162 non-null  float64
 5   km                     462162 non-null  object 
 6   city_name              463152 non-null  object 
 7   primary_cause          463152 non-null  object 
 8   accident_type          463152 non-null  object 
 9   casualty_status        463152 non-null  object 
 10  daylight_condition     463152 non-null  object 
 11  traffic_direction      463152 non-null  object 
 12  weather_condition      463152 non-null  object 
 13  highway_type           463152 non-null  object 
 14  road_geometry          463152 non-nu

In [46]:
df_renamed.dtypes

accident_date             object
day_of_week               object
accident_time             object
state_code                object
highway_number           float64
km                        object
city_name                 object
primary_cause             object
accident_type             object
casualty_status           object
daylight_condition        object
traffic_direction         object
weather_condition         object
highway_type              object
road_geometry             object
total_people_involved      int64
fatalities                 int64
minor_injuries             int64
serious_injuries           int64
uninjured_count            int64
unknown_status             int64
total_casualties           int64
vehicle_count              int64
accident_latitude        float64
accident_longitude       float64
regional_office           object
reporting_station         object
dtype: object

In [47]:
def fix_data_types(df):
    """Fix data types and handle inconsistencies"""
    
    # Convert accident_date to datetime
    # YOUR CODE HERE: Convert 'accident_date' to datetime format
    df['accident_date'] = pd.to_datetime(df['accident_date'])
    df['accident_time'] = pd.to_datetime(df['accident_time'], format='%H:%M:%S', errors='coerce')

    
    # Extract hour from accident_time or convert to integer hour
    # YOUR CODE HERE: Convert 'accident_time' to integer hour and store as 'accident_hour'
    df['accident_hour'] = df['accident_time'].dt.hour

    # Convert kilometer_marker to numeric (handle "43,4" format)
    # YOUR CODE HERE: Convert 'kilometer_marker' to numeric, replacing commas with dots
    # hint: consider format='%H:%M:%S', errors='coerce' when using pd.to_datetime
    df['km'] = pd.to_numeric(df['km'].str.replace(',', '.'))

    # Standardize text fields (lowercase, strip whitespace)
    # YOUR CODE HERE: Define text columns to standardize
    text_columns = [
        'day_of_week', 
        'state_code', 'city_name', 'primary_cause', 'accident_type',
        'casualty_status', 'daylight_condition', 'traffic_direction',
        'weather_condition', 'highway_type', 'road_geometry', 
        'regional_office', 'reporting_station'
    ]
    
    # Example of text columns to standardize
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].str.lower().str.strip()
             # YOUR CODE HERE: Standardize text columns to lowercase and strip whitespace
    
    print("✓ Data types fixed and standardized")
    return df

In [48]:
df_renamed_fixed = fix_data_types(df_renamed)
df_renamed_fixed.dtypes

✓ Data types fixed and standardized


accident_date            datetime64[ns]
day_of_week                      object
accident_time            datetime64[ns]
state_code                       object
highway_number                  float64
km                              float64
city_name                        object
primary_cause                    object
accident_type                    object
casualty_status                  object
daylight_condition               object
traffic_direction                object
weather_condition                object
highway_type                     object
road_geometry                    object
total_people_involved             int64
fatalities                        int64
minor_injuries                    int64
serious_injuries                  int64
uninjured_count                   int64
unknown_status                    int64
total_casualties                  int64
vehicle_count                     int64
accident_latitude               float64
accident_longitude              float64


In [49]:
df_renamed_fixed.head()

,accident_date,day_of_week,accident_time,state_code,highway_number,km,city_name,primary_cause,accident_type,casualty_status,...,serious_injuries,uninjured_count,unknown_status,total_casualties,vehicle_count,accident_latitude,accident_longitude,regional_office,reporting_station,accident_hour
0,2017-01-01,sunday,1900-01-01 01:45:00,rs,116.0,34.9,vacaria,mechanical loss/defect of vehicle,rear-end collision,with injured victims,...,0,2,0,4,2,-28.50712,-50.94118,sprf-rs,del05-rs,1
1,2017-01-01,sunday,1900-01-01 01:00:00,pr,376.0,636.0,tijucas do sul,incompatible velocity,run-off-road,with dead victims,...,0,1,0,0,2,-25.75400,-49.12660,sprf-pr,del01-pr,1
2,2017-01-01,sunday,1900-01-01 04:40:00,ba,101.0,65.0,entre rios,driver was sleeping,head-on collision,with dead victims,...,1,2,0,2,2,-11.96180,-38.09530,sprf-ba,del01-ba,4
3,2017-01-01,sunday,1900-01-01 06:30:00,pa,316.0,72.5,castanhal,driver's lack of attention to conveyance,side impact collision,with dead victims,...,0,3,0,0,3,-1.28998,-47.83483,sprf-pa,del01-pa,6
4,2017-01-01,sunday,1900-01-01 09:00:00,go,20.0,220.5,posse,road's defect,collision with fixed object,with injured victims,...,1,0,0,3,1,-14.14221,-46.32259,sprf-df,del02-df,9


In [50]:
def perform_data_quality_checks(df):
    """Comprehensive data quality assessment"""
    
    print("=== DATA QUALITY REPORT ===")
    
    # 1. Missing Values Documentation
    # YOUR CODE HERE: Count missing values in each column
    missing_counts = df.isnull().sum()
    total_rows = len(df)

    # if there are missing data, loop through each column and print the number of missing values and percentage
    # percentage is calculated as (missing_count / total_rows) * 100
    print("Missing Values Percentage:")
    for col in df.columns:
        count = missing_counts[col]
        percentage_missing = (count / total_rows) * 100
        if percentage_missing > 0.05:
            print(f"{col} missing {percentage_missing:.02}%")
    
    
    # 2. Remove rows with missing data
    # YOUR CODE HERE: Remove rows with any missing values
    df_clean = df.dropna()
    removed_rows = total_rows
    print(f"\n   Removed {removed_rows} rows with missing data")
    
    # 3. Outlier Detection
    print("\n2. OUTLIER ANALYSIS:")
    numerical_cols = ['total_people_involved', 'fatalities', 'minor_injuries', 
                     'serious_injuries', 'total_casualties', 'vehicle_count']
    
    outlier_summary = {}
    # Outlier detection is done for you using the IQR method
    # THIS IS A SIMLPE METHOD AND MAY NOT BE SUITABLE FOR ALL DATASETS 
    # YOU WILL LEARN ABOUT IQR IN A LATER LESSON
    for col in numerical_cols:
        if col in df_clean.columns:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)]
            outlier_summary[col] = {
                'count': len(outliers),
                'percentage': (len(outliers) / len(df_clean)) * 100,
                'max_value': df_clean[col].max(),
                'realistic_max': upper_bound
            }
    
    for col, stats in outlier_summary.items():
        print(f"   {col}: {stats['count']} outliers ({stats['percentage']:.2f}%)")
        print(f"      Max value: {stats['max_value']}, Expected max: {stats['realistic_max']:.1f}")
    
    # 4. Data Quality Check: total_injured calculation
    print("\n3. DATA CONSISTENCY CHECK:")
    # YOUR CODE HERE: Calculate total casualties as sum of fatalities, minor injuries, and serious injuries
    df_clean['calculated_total'] = df['fatalities'] + df['minor_injuries'] + df['serious_injuries']

    # YOUR CODE HERE: Check if calculated total matches total_casualties
    df_clean['total_match_ind'] = df_clean['calculated_total'] == df_clean['total_casualties']
    
    mismatches = (~df_clean['total_match_ind']).sum()
    print(f"   Total casualties mismatch: {mismatches} records ({(mismatches/len(df_clean)*100):.2f}%)")
    
    return df_clean

In [51]:
df_clean = perform_data_quality_checks(df_renamed_fixed)
df_clean.head()

=== DATA QUALITY REPORT ===
Missing Values Percentage:
highway_number missing 0.21%
km missing 0.21%
reporting_station missing 0.28%

   Removed 463152 rows with missing data

2. OUTLIER ANALYSIS:
   total_people_involved: 8545 outliers (1.85%)
      Max value: 80, Expected max: 6.0
   fatalities: 31212 outliers (6.77%)
      Max value: 21, Expected max: 0.0
   minor_injuries: 20205 outliers (4.38%)
      Max value: 61, Expected max: 2.5
   serious_injuries: 96315 outliers (20.90%)
      Max value: 31, Expected max: 0.0
   total_casualties: 30354 outliers (6.59%)
      Max value: 66, Expected max: 2.5
   vehicle_count: 8558 outliers (1.86%)
      Max value: 23, Expected max: 3.5

3. DATA CONSISTENCY CHECK:
   Total casualties mismatch: 31212 records (6.77%)


,accident_date,day_of_week,accident_time,state_code,highway_number,km,city_name,primary_cause,accident_type,casualty_status,...,unknown_status,total_casualties,vehicle_count,accident_latitude,accident_longitude,regional_office,reporting_station,accident_hour,calculated_total,total_match_ind
0,2017-01-01,sunday,1900-01-01 01:45:00,rs,116.0,34.9,vacaria,mechanical loss/defect of vehicle,rear-end collision,with injured victims,...,0,4,2,-28.50712,-50.94118,sprf-rs,del05-rs,1,4,True
1,2017-01-01,sunday,1900-01-01 01:00:00,pr,376.0,636.0,tijucas do sul,incompatible velocity,run-off-road,with dead victims,...,0,0,2,-25.75400,-49.12660,sprf-pr,del01-pr,1,1,False
2,2017-01-01,sunday,1900-01-01 04:40:00,ba,101.0,65.0,entre rios,driver was sleeping,head-on collision,with dead victims,...,0,2,2,-11.96180,-38.09530,sprf-ba,del01-ba,4,3,False
3,2017-01-01,sunday,1900-01-01 06:30:00,pa,316.0,72.5,castanhal,driver's lack of attention to conveyance,side impact collision,with dead victims,...,0,0,3,-1.28998,-47.83483,sprf-pa,del01-pa,6,1,False
4,2017-01-01,sunday,1900-01-01 09:00:00,go,20.0,220.5,posse,road's defect,collision with fixed object,with injured victims,...,0,3,1,-14.14221,-46.32259,sprf-df,del02-df,9,3,True


In [52]:
df_clean.dtypes

accident_date            datetime64[ns]
day_of_week                      object
accident_time            datetime64[ns]
state_code                       object
highway_number                  float64
km                              float64
city_name                        object
primary_cause                    object
accident_type                    object
casualty_status                  object
daylight_condition               object
traffic_direction                object
weather_condition                object
highway_type                     object
road_geometry                    object
total_people_involved             int64
fatalities                        int64
minor_injuries                    int64
serious_injuries                  int64
uninjured_count                   int64
unknown_status                    int64
total_casualties                  int64
vehicle_count                     int64
accident_latitude               float64
accident_longitude              float64


In [53]:
def create_engineered_features(df):
    """Create new features for enhanced analysis"""
    
    # Extract date components
    # YOUR CODE HERE: Extract year, month, and quarter from 'accident_date'
    df['year'] = df['accident_date'].dt.year
    df['month'] = df['accident_date'].dt.month
    df['day'] = df['accident_date'].dt.day
    df['quarter'] = df['accident_date'].dt.quarter
    
    # Create accident severity score
    # YOUR CODE HERE: Define a scoring system based on accident severity call it 'accident_severity_score'
    df['accident_severity_score'] = (
        (df['fatalities'] * 10) + (df['serious_injuries'] * 3) + (df['minor_injuries'])
    )

    # Time of day categories function created for you to use
    def categorize_time(hour):
        if pd.isna(hour):
            return 'unknown'
        elif 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 18:
            return 'afternoon'
        elif 18 <= hour < 22:
            return 'evening'
        else:
            return 'night'
    
    #   YOUR CODE HERE: Apply categorize_time function to 'accident_time' 
    df['time_of_day_category'] = df['accident_hour'].apply(categorize_time)
    
    # Weekend flag
    # YOUR CODE HERE: Check if 'day_of_week' is Saturday or Sunday
    df['weekend_flag'] = df['day_of_week'].isin(['saturday', 'sunday'])
    
    # Fatal accident flag
    # YOUR CODE HERE: Check if 'fatalities' > 0
    df['fatal_accident_flag'] = df['fatalities'] > 0
    
    # Multi-vehicle flag
    # YOUR CODE HERE: Check if 'vehicle_count' > 1
    df['multi_vehicle_flag'] = df['vehicle_count'] > 1
    
    print("✓ Feature engineering completed")
    print(f"   Created {len(['year', 'month', 'quarter', 'accident_severity_score', 'time_of_day_category', 'weekend_flag', 'fatal_accident_flag', 'multi_vehicle_flag'])} new features")
    
    return df

In [54]:
df_featured = create_engineered_features(df_clean)
df_featured.head()

✓ Feature engineering completed
   Created 8 new features


,accident_date,day_of_week,accident_time,state_code,highway_number,km,city_name,primary_cause,accident_type,casualty_status,...,total_match_ind,year,month,day,quarter,accident_severity_score,time_of_day_category,weekend_flag,fatal_accident_flag,multi_vehicle_flag
0,2017-01-01,sunday,1900-01-01 01:45:00,rs,116.0,34.9,vacaria,mechanical loss/defect of vehicle,rear-end collision,with injured victims,...,True,2017,1,1,1,4,night,True,False,True
1,2017-01-01,sunday,1900-01-01 01:00:00,pr,376.0,636.0,tijucas do sul,incompatible velocity,run-off-road,with dead victims,...,False,2017,1,1,1,10,night,True,True,True
2,2017-01-01,sunday,1900-01-01 04:40:00,ba,101.0,65.0,entre rios,driver was sleeping,head-on collision,with dead victims,...,False,2017,1,1,1,14,night,True,True,True
3,2017-01-01,sunday,1900-01-01 06:30:00,pa,316.0,72.5,castanhal,driver's lack of attention to conveyance,side impact collision,with dead victims,...,False,2017,1,1,1,10,morning,True,True,True
4,2017-01-01,sunday,1900-01-01 09:00:00,go,20.0,220.5,posse,road's defect,collision with fixed object,with injured victims,...,True,2017,1,1,1,5,morning,True,False,False


In [55]:
df_featured.columns

Index(['accident_date', 'day_of_week', 'accident_time', 'state_code',
       'highway_number', 'km', 'city_name', 'primary_cause', 'accident_type',
       'casualty_status', 'daylight_condition', 'traffic_direction',
       'weather_condition', 'highway_type', 'road_geometry',
       'total_people_involved', 'fatalities', 'minor_injuries',
       'serious_injuries', 'uninjured_count', 'unknown_status',
       'total_casualties', 'vehicle_count', 'accident_latitude',
       'accident_longitude', 'regional_office', 'reporting_station',
       'accident_hour', 'calculated_total', 'total_match_ind', 'year', 'month',
       'day', 'quarter', 'accident_severity_score', 'time_of_day_category',
       'weekend_flag', 'fatal_accident_flag', 'multi_vehicle_flag'],
      dtype='object')

In [56]:
df_featured.info()

<class 'pandas.core.frame.DataFrame'>
Index: 460929 entries, 0 to 463151
Data columns (total 39 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   accident_date            460929 non-null  datetime64[ns]
 1   day_of_week              460929 non-null  object        
 2   accident_time            460929 non-null  datetime64[ns]
 3   state_code               460929 non-null  object        
 4   highway_number           460929 non-null  float64       
 5   km                       460929 non-null  float64       
 6   city_name                460929 non-null  object        
 7   primary_cause            460929 non-null  object        
 8   accident_type            460929 non-null  object        
 9   casualty_status          460929 non-null  object        
 10  daylight_condition       460929 non-null  object        
 11  traffic_direction        460929 non-null  object        
 12  weather_condition    

# Create Four Targeted Analysis DataFrames

## Analysis 1: Severity Analysis - What Conditions Lead to the Most Severe Accidents? 🚨
This analysis uses the powerful `accident_severity_score` to find out which conditions are associated with more dangerous accidents, not just more frequent ones.
* __Purpose__: Identify which weather and road combinations produce the most dangerous accidents (not just frequent ones)
* __Analysis__: Group by `weather_condition` and `road_classification`. Calculate the average `accident_severity_score` for each combination.
* __Resulting DataFrame Columns__: `weather_condition`, `road_classification`, `average_severity_score`

In [57]:
def create_severity_analysis(df):
    """
    Analysis 1: Severity Analysis
    Find which conditions lead to the most severe accidents using accident_severity_score
    """
    
    # Group by weather and road conditions to calculate average severity score and count accidents
    # YOUR CODE HERE
    severity_df = df.groupby(by=['weather_condition', 'highway_type']).agg({'accident_severity_score' : 'mean',
                                                                                   'accident_date': 'count'})
    
    # rename columns for clarity
    severity_df.columns = ['average_severity_score', 'accident_count']
    
    # Reset index to make grouping columns regular columns
    severity_df.reset_index(inplace=True)
    
    # Sort by severity score to identify most dangerous combinations
    severity_df.sort_values(by='average_severity_score', ascending=False, inplace=True)
    
    # Final DataFrame with required columns
    # YOUR CODE HERE: Create a DataFrame with 'weather_condition', 'road_classification', and 'average_severity_score'
    severity_analysis = severity_df[['weather_condition', 'highway_type', 'average_severity_score']]
    
    print("✓ Severity Analysis completed")
    print(f"   Found {len(severity_analysis)} weather/road combinations")
    print(f"   Most severe combination: {severity_analysis.iloc[0]['weather_condition']} + {severity_analysis.iloc[0]['highway_type']}")
    
    return severity_analysis

In [58]:
severity_analysis = create_severity_analysis(df_featured)
severity_analysis.head()

✓ Severity Analysis completed
   Found 28 weather/road combinations
   Most severe combination: fog + simple


,weather_condition,highway_type,average_severity_score
11,fog,simple,3.514563
16,ignored,simple,3.272149
27,windy,simple,3.037647
19,rainy,simple,2.980007
2,clear sky,simple,2.930011


## Analysis 2: Temporal Patterns of Fatal Accidents - When are Fatal Accidents Most Common? 🌃
This analysis focuses specifically on the most tragic outcomes using the new flags.
* __Purpose__: Focus specifically on fatal accidents to understand when the most tragic outcomes occur
* __Analysis__: Filter the data for `fatal_accident_flag` == True. Then, group by `time_of_day_category` and `weekend_flag` and count the number of fatal accidents.
* __Resulting DataFrame Columns__: `time_of_day_category`, `weekend_flag`, `number_of_fatal_accidents`

In [59]:
def create_fatal_temporal_analysis(df):
    """
    Analysis 2: Temporal Patterns of Fatal Accidents
    Filter for fatal accidents and analyze timing patterns
    """
    
    # Filter for fatal accidents only
    # YOUR CODE HERE: Filter DataFrame for fatal accidents where 'fatal_accident_flag' is True
    fatal_accidents = df[df['fatal_accident_flag']]
    # YOUR CODE HERE: Check if there are any fatal accidents, if there are no fatal accident print "⚠️ No fatal accidents found in dataset"
    if len(fatal_accidents) == 0:
        print("No fatal accidents found in dataset")
        return 

    # Group by time and weekend patterns (time_of_day_category and weekend_flag) and capture the count of fatal accidents
    fatal_temporal_df = fatal_accidents.groupby(['time_of_day_category', 'weekend_flag']).agg({'fatal_accident_flag': 'count'}).reset_index()

    # rename columns for clarity
    fatal_temporal_df.columns = ['time_of_day_category', 'weekend_flag', 'number_of_fatal_accidents']
    
    # Sort by number of fatal accidents for insights
    # YOUR CODE HERE: Sort the DataFrame by 'number_of_fatal_accidents' in descending order
    fatal_temporal_df = fatal_temporal_df.sort_values(by='number_of_fatal_accidents', ascending=False)
    
    print("✓ Fatal Temporal Analysis completed")
    print(f"   Total fatal accidents analyzed: {len(fatal_accidents)}")
    print(f"   Peak fatal accident period: {fatal_temporal_df.iloc[0]['time_of_day_category']} ({'weekend' if fatal_temporal_df.iloc[0]['weekend_flag'] else 'weekday'})")
    
    return fatal_temporal_df

In [60]:
fatal_temporal_analysis = create_fatal_temporal_analysis(df_featured)
fatal_temporal_analysis.head()

✓ Fatal Temporal Analysis completed
   Total fatal accidents analyzed: 31212
   Peak fatal accident period: evening (weekday)


,time_of_day_category,weekend_flag,number_of_fatal_accidents
2,evening,False,5421
6,night,False,4874
0,afternoon,False,4676
7,night,True,4014
4,morning,False,4012


## Analysis 3: Accident Cause and Vehicle Involvement 🚗
This analysis explores the relationship between the primary cause of an accident and whether it involved single or multiple vehicles.
* __Purpose__: Explore relationship between accident causes and single vs multi-vehicle involvement
* __Analysis__: Group by accident_cause and `multi_vehicle_flag`. Count the number of accidents and also calculate the average accident_severity_score.
* __Resulting DataFrame Columns__: `accident_cause`, `multi_vehicle_flag`, `number_of_accidents`, `average_severity_score`


In [61]:
def create_cause_vehicle_analysis(df):
    """
    Analysis 3: Accident Cause and Vehicle Involvement
    Analyze relationship between causes and multi-vehicle involvement
    """
    
    # Group by accident cause and vehicle involvement (accident_cause and multi_vehicle_flag) and calculate average severity score and count accidents
    cause_vehicle_df = df.groupby(['primary_cause', 'multi_vehicle_flag']).agg({
        'accident_date': 'count',
        'accident_severity_score': 'mean'
    })

    # rename columns for clarity
    cause_vehicle_df.columns = ['number_of_accidents', 'average_severity_score']
    
    # Reset index to make grouping columns regular columns
    cause_vehicle_df = cause_vehicle_df.reset_index()
    
    # Sort by number of accidents for primary insights
    cause_vehicle_df = cause_vehicle_df.sort_values(by='number_of_accidents', ascending=False)
    
    print("✓ Cause-Vehicle Analysis completed")
    print(f"   Analyzed {len(cause_vehicle_df)} cause/vehicle combinations")
    
    # Show most common cause for single vs multi-vehicle
    single_vehicle = cause_vehicle_df.loc[cause_vehicle_df['multi_vehicle_flag'] == False]
    multi_vehicle = cause_vehicle_df.loc[cause_vehicle_df['multi_vehicle_flag'] == True]
    
    if len(single_vehicle) > 0:
        print(f"   Most common single-vehicle cause: {single_vehicle.iloc[0]['primary_cause']}")
    if len(multi_vehicle) > 0:
        print(f"   Most common multi-vehicle cause: {multi_vehicle.iloc[0]['primary_cause']}")
    
    return cause_vehicle_df


In [62]:
cause_vehicle_analysis = create_cause_vehicle_analysis(df_featured)
cause_vehicle_analysis.head()

✓ Cause-Vehicle Analysis completed
   Analyzed 167 cause/vehicle combinations
   Most common single-vehicle cause: driver's lack of attention to conveyance
   Most common multi-vehicle cause: driver's lack of attention to conveyance


,primary_cause,multi_vehicle_flag,number_of_accidents,average_severity_score
55,driver's lack of attention to conveyance,True,70972,2.247436
54,driver's lack of attention to conveyance,False,36583,1.787005
80,incompatible velocity,False,31471,2.185695
37,driver broke the laws of transit,True,25790,2.988174
56,driver's lack of reaction,False,20662,2.054303


## Analysis 4: Quarterly Accident Hotspot Analysis 📍
This analysis uses the location and time data to find seasonal accident hotspots.
* __Purpose__: Identify seasonal accident patterns by state to find regional hotspots
* __Analysis__: Group by `state_code` and quarter. Count the number of accidents to identify if certain states have accident seasons.
* __Resulting DataFrame Columns__: `state_code`, `quarter`, `total_accidents`

In [63]:
def create_quarterly_hotspot_analysis(df):
    """
    Analysis 4: Quarterly Accident Hotspot Analysis
    Find seasonal accident hotspots by state
    """
    
    # group by state_code and quarter to count accidents
    # reset index to make grouping columns regular columns
    # YOUR CODE
    hotspot_df = df.groupby(by=['state_code', 'quarter']).agg({'accident_date' : 'count'}).reset_index()
    
    # rename columns for clarity
    hotspot_df.columns = ['state_code', 'quarter', 'total_accidents']
    
    # Sort by total accidents to identify hotspots
    hotspot_df = hotspot_df.sort_values(by='total_accidents', ascending=False)
    
    # Add quarter labels for better interpretation
    # create a mapping for quarter numbers to names for example {1: 'Q1 (Jan-Mar)'}
    quarter_labels = {1: 'Q1 (Jan-Mar)', 2: 'Q2 (Apr-June)', 3: 'Q3 (July-Sept), 4: (Oct-Dec)'}
    hotspot_df['quarter_name'] = hotspot_df['quarter'].map(quarter_labels)
    
    print("✓ Quarterly Hotspot Analysis completed")
    print(f"   Analyzed {len(df['state_code'].unique())} states across 4 quarters")
    
    # Identify top hotspot
    # YOUR CODE: Get the row with the highest total accidents
    top_hotspot = hotspot_df.iloc[0]
    print(f"   Top seasonal hotspot: {top_hotspot['state_code']} in {top_hotspot['quarter_name']} ({top_hotspot['total_accidents']} accidents)")
    
    # Final DataFrame with required columns (removing quarter_name as it's just for logging)
    final_hotspot_df = hotspot_df[['state_code', 'quarter', 'total_accidents']].copy()
    
    return final_hotspot_df

In [64]:
quarterly_hotspot_analysis = create_quarterly_hotspot_analysis(df_featured)
quarterly_hotspot_analysis.head()

✓ Quarterly Hotspot Analysis completed
   Analyzed 27 states across 4 quarters
   Top seasonal hotspot: mg in Q1 (Jan-Mar) (16230 accidents)


,state_code,quarter,total_accidents
40,mg,1,16230
42,mg,3,15146
43,mg,4,14961
41,mg,2,14829
92,sc,1,14463


## Complete Data Processing Pipeline (putting it all together)

In [65]:
def main_analysis_pipeline():
    """
    Complete pipeline from S3 to Snowflake with four targeted analyses
    """
    
    print("🚀 Starting UK Traffic Accident Analysis Pipeline")
    print("=" * 60)
    
    # Step 1: Load data
    df = load_accident_data()
    
    # Step 2: Rename columns
    df =  rename_columns(df)
    
    # Step 3: Fix data types
    df =  fix_data_types(df)
    
    # Step 4: Data quality checks
    df_clean = perform_data_quality_checks(df)
    
    # Step 5: Feature engineering
    df_final = create_engineered_features(df)
    
    print("\n🔍 Creating Four Targeted Analyses")
    print("=" * 40)
    
    # Step 6: Create four analysis DataFrames using the df_final DataFrame
    analysis_1 =  create_severity_analysis(df_final)
    analysis_2 = create_fatal_temporal_analysis(df_final)
    analysis_3 = create_cause_vehicle_analysis(df_final)
    analysis_4 = create_quarterly_hotspot_analysis(df_final)
    
    return analysis_1, analysis_2, analysis_3, analysis_4, df_final

# Run the complete pipeline
severity_df, fatal_temporal_df, cause_vehicle_df, hotspot_df, cleaned_data = main_analysis_pipeline()

🚀 Starting UK Traffic Accident Analysis Pipeline


Loaded 463152 records with 27 columns
✓ Columns renamed successfully
✓ Data types fixed and standardized
=== DATA QUALITY REPORT ===
Missing Values Percentage:
highway_number missing 0.21%
km missing 0.21%
reporting_station missing 0.28%

   Removed 463152 rows with missing data

2. OUTLIER ANALYSIS:
   total_people_involved: 8545 outliers (1.85%)
      Max value: 80, Expected max: 6.0
   fatalities: 31212 outliers (6.77%)
      Max value: 21, Expected max: 0.0
   minor_injuries: 20205 outliers (4.38%)
      Max value: 61, Expected max: 2.5
   serious_injuries: 96315 outliers (20.90%)
      Max value: 31, Expected max: 0.0
   total_casualties: 30354 outliers (6.59%)
      Max value: 66, Expected max: 2.5
   vehicle_count: 8558 outliers (1.86%)
      Max value: 23, Expected max: 3.5

3. DATA CONSISTENCY CHECK:
   Total casualties mismatch: 31212 records (6.77%)
✓ Feature engineering completed
   Created 8 new features

🔍 Creating Four Targeted Analyses
✓ Severity Analysis completed
   F

# Snowflake Integration
* Push all four datasets into Snowflake and save them as tables (transient or standard)

## Four Snowflake Temporary Tables:

* __SEVERITY_ANALYSIS__:
    * Columns: weather_condition, road_classification, average_severity_score
    * Purpose: Risk assessment for infrastructure and safety planning


* __FATAL_TEMPORAL_PATTERNS__
    * Columns: time_of_day_category, weekend_flag, number_of_fatal_accidents
    * Purpose: Emergency service resource allocation and enforcement timing


* __CAUSE_VEHICLE_ANALYSIS__:
    * Columns: accident_cause, multi_vehicle_flag, number_of_accidents, average_severity_score
    * Purpose: Prevention strategy development and driver education programs

* __QUARTERLY_HOTSPOT_ANALYSIS__:
    * Columns: state_code, quarter, total_accidents
    * Purpose: Seasonal safety campaign planning and resource allocation

In [66]:
# Your code to save the DataFrames to Snowflake or any other destination
# use a snow.cfg file to configure the connection
# call the section [DEV]

import pandas as pd
from snowflake import connector
from configparser import ConfigParser

config = ConfigParser()
config.read('snow.cfg')

config.sections()

['READ', 'WRITE']

In [67]:
params = dict(config['WRITE'])

In [68]:
# After you write your data, write SQL queries to verify the data
# For example, you can use the following SQL query to check the number of records in each 
# SELECT COUNT(*) FROM <your_table_name>; for each of the DataFrames you created
from snowflake.snowpark import Session

session = Session.builder.configs(params).create()

snow_severity_df = session.create_dataframe(severity_df)
snow_fatal_temporal_df = session.create_dataframe(fatal_temporal_df)
snow_cause_vehicle_df = session.create_dataframe(cause_vehicle_df)
snow_hotspot_df = session.create_dataframe(hotspot_df)

snow_severity_df.write.mode('overwrite').save_as_table('SEVERITY_ANALYSIS')
snow_fatal_temporal_df.write.mode('overwrite').save_as_table('FATAL_TEMPORAL_ANALYSIS')
snow_cause_vehicle_df.write.mode('overwrite').save_as_table('CAUSE_VEHICLE_ANALYSIS')
snow_hotspot_df.write.mode('overwrite').save_as_table('QUARTERLY_HOTSPOT_ANALYSIS')
